## Setup

In [1]:
import os

import pandas as pd
import numpy as np


project_path = os.path.join('/ubc/ece/home/so/grads/joshchiu/', 'sse-532v', 'llm')
os.chdir(project_path)

df = pd.read_csv('emails_cleaned.csv')
df.head(5)

,file,message,content,Message-ID,Date,From,To,Subject,Cc,Mime-Version,...,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,has_other_content,if_forwarded
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n,<18782981.1075855378110.JavaMail.evans@thyme,"Mon, 14 May 2001 16:39:00 -0700 (PDT",phillip.allen@enron.co,tim.belden@enron.co,NaN,NaN,1.0,...,NaN,Phillip K Alle,Tim Belden <Tim Belden/Enron@EnronXGate,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-,pallen (Non-Privileged).pst,False,False
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,<15464986.1075855378456.JavaMail.evans@thyme,"Fri, 4 May 2001 13:51:00 -0700 (PDT",phillip.allen@enron.co,john.lavorato@enron.co,Re,NaN,1.0,...,NaN,Phillip K Alle,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-,pallen (Non-Privileged).pst,False,False
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,<24216240.1075855687451.JavaMail.evans@thyme,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT",phillip.allen@enron.co,leah.arsdall@enron.co,Re: tes,NaN,1.0,...,NaN,Phillip K Alle,Leah Van Arsdal,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mai,Allen-,pallen.nsf,False,False
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...",<13505866.1075863688222.JavaMail.evans@thyme,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT",phillip.allen@enron.co,randall.gay@enron.co,NaN,NaN,1.0,...,NaN,Phillip K Alle,Randall L Ga,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mai,Allen-,pallen.nsf,False,False
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,<30922949.1075863688243.JavaMail.evans@thyme,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT",phillip.allen@enron.co,greg.piper@enron.co,Re: Hell,NaN,1.0,...,NaN,Phillip K Alle,Greg Pipe,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mai,Allen-,pallen.nsf,False,False


In [2]:
# Filter for sent folder
df_sent = df[df["file"].str.contains("_sent_mail", na=False)]
df_sent = df_sent[['content']]
df_sent = df_sent.dropna(subset=["content"])
df_sent.head(5)

,content
0,Here is our forecast\n\n
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,\n\n Can you send me a schedule of the s..."
4,Let's shoot for Tuesday at 11:45.


In [3]:
## Global Arguments

MODEL_NAME = "gpt-4o-mini"
SEED = 42
DOCUMENT_COUNT_TO_ATTACKER = 100
DOCUMENT_COUNT_TO_CLIENT = 500

GENERATOR_TARGET_SIZE = DOCUMENT_COUNT_TO_CLIENT - DOCUMENT_COUNT_TO_ATTACKER

OPENAI_API_KEY = ""


In [4]:
import pickle
working_dir = os.getcwd()

def load_object(name: str):
    if not os.path.exists(name):
        raise ValueError(f"The file {name} does not exist")

    print(f"load_object: loading object {name}")
    with open(name, "rb") as f:
        object = pickle.load(f)
    return object

def save_object(object, name: str, overwrite=False) -> bool:
    print(f"save_object: saving object {name}")
    with open(name, 'wb') as f:
        pickle.dump(object, f)

### Split Dataset

In [5]:
# Shuffle and split in 2

split_point = len(df_sent) // 2
df_shuffled = df_sent.sample(frac=1, random_state=SEED).reset_index(drop=True)
df_client = df_shuffled.iloc[:split_point].reset_index(drop=True)
df_attacker = df_shuffled.iloc[split_point:].reset_index(drop=True)

df_attacker_sampled = df_attacker.sample(n=DOCUMENT_COUNT_TO_ATTACKER, random_state=SEED).reset_index(drop=True)
df_client_sampled = df_client.sample(n=DOCUMENT_COUNT_TO_CLIENT, random_state=SEED).reset_index(drop=True)



attacker_documents = list(map(str, df_attacker_sampled['content'].tolist()))
client_documents = list(map(str, df_client_sampled['content'].tolist()))

print(f"Client documents: {len(client_documents)}")
print(f"Attacker documents: {len(attacker_documents)}")

df_attacker_sampled.head(5)

Client documents: 500
Attacker documents: 100


,content
0,"Well, how does it feel to be back? I'm sure, ..."
1,Mike Sandridge and I finally spoke on the phon...
2,"I had a vegetarian sandwich.\n\n\n\n\n""Neil Ma..."
3,"Pat, \n\nI think that we should probably send ..."
4,"Jerry, I have not cancelled the Station 4 air ..."


## Documents Generation

In [6]:
from typing import List
import numpy as np
import csv
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, fcluster
import openai
import math

openai.client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Logger using print statements
def log(message):
    print(f"generate_documents: {message}")

# Append generated documents to CSV
def append_to_csv(filename, documents):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for doc in documents:
            writer.writerow([doc])

def generate_documents(documents: List[str], target: int, output_csv: str) -> List[str]:
    """
    Generate more documents in the same style based on an input list of documents. Return a list of all new documents including the argument.

    Keyword arguments:
    documents -- the list of reference documents
    target -- the target number of documents including the list given
    output_csv -- file path to save generated documents incrementally
    """

    if not documents or target <= len(documents):
        log("No new documents need to be generated.")
        return documents  # No need to generate new ones

    log("Starting document generation process...")

    # Step 1: Vectorize text
    log("Vectorizing input documents...")
    vectorizer = TfidfVectorizer(stop_words='english', max_features=300)
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Step 2: Compute distances & perform clustering
    log("Computing cosine distance matrix and performing clustering...")
    distance_matrix = pdist(tfidf_matrix.toarray(), metric='euclidean')
    linkage_matrix = linkage(distance_matrix, method='ward')

    # Step 3: Assign clusters using a distance threshold
    merge_distances = linkage_matrix[:, 2]
    distance_threshold = np.percentile(merge_distances, 65)
    clusters = fcluster(linkage_matrix, distance_threshold, criterion='distance')
    log(f"Assigned {len(set(clusters))} clusters.")

    # Step 4: Generate new documents from each cluster
    cluster_dict = {}
    for i, cluster in enumerate(clusters):
        if cluster not in cluster_dict:
            cluster_dict[cluster] = []
        cluster_dict[cluster].append(documents[i])

    new_documents = []
    missing_count = target - len(documents)

    message_per_cluster = math.ceil(missing_count / len(set(clusters)))

    while len(new_documents) < target:
        for selected_cluster in sorted(cluster_dict.keys()):  # Process clusters sequentially
          if missing_count <= 0:
              break

          sample_size = min(3, len(cluster_dict[selected_cluster]))  # Take 2-3 samples
          examples = cluster_dict[selected_cluster][:sample_size]  # Take sequential samples

          log(f"Generating {message_per_cluster} new texts based on cluster {selected_cluster} ({len(new_documents)}/{target})...")

          # Generate new text using GPT
          prompt = f"Generate {message_per_cluster} new texts similar to the following examples:\n\n"
          prompt += '\n\n'.join(f'Example {i+1}: {doc}' for i, doc in enumerate(examples))
          prompt += "\n\nReturn only the response as a separate entry and put exactly '###' between them so i can parse them."

          retries = 3
          for attempt in range(retries):
              try:
                  response = openai.client.chat.completions.create(
                      model=MODEL_NAME,
                      messages=[{"role": "system", "content": "You are an AI that generates text similar to given examples."},
                                {"role": "user", "content": prompt}],
                      max_tokens=500
                  )
                  message = response.choices[0].message.content
                  if message:
                      generated_texts = message.split('###')
                      new_documents.extend(generated_texts)
                      missing_count -= len(generated_texts)
                      log(f"Generated {len(generated_texts)} new documents.")

                      # Append progress to CSV after each response
                      append_to_csv(output_csv, generated_texts)
                      log(f"Appended progress to {output_csv}.")
                      break  # Exit retry loop on success
                  else:
                      log("No valid choices received from OpenAI response.")
              except Exception as e:
                  log(f"Error during OpenAI API call: {e}")
                  if attempt < retries - 1:
                      wait_time = 2 ** attempt  # Exponential backoff
                      log(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                      time.sleep(wait_time)
                  else:
                      break
          if len(new_documents) >= target:
            break


    log("Document generation process completed.")
    return new_documents[:target]  # Ensure we don't exceed the target


In [ ]:
name = f"{MODEL_NAME}_{DOCUMENT_COUNT_TO_CLIENT}client_{DOCUMENT_COUNT_TO_ATTACKER}attacker_{GENERATOR_TARGET_SIZE}target.pkl"

dir = os.path.join(working_dir, "generation_working_dir")
os.makedirs(dir, exist_ok=True)


try:
    generated_documents = load_object(os.path.join(dir, name))
    print("Loaded attacker documents from file")
except ValueError:
    temp_file = os.path.join(dir, "generation.csv")
    print(f"Generating attacker {GENERATOR_TARGET_SIZE} documents at {temp_file}")
    open(temp_file, 'w').close()
    generated_documents = generate_documents(attacker_documents, GENERATOR_TARGET_SIZE, temp_file)  # Save to file
    save_object(generated_documents, os.path.join(dir, name))

print(f"""Attacker documents: {len(attacker_documents)} total (original)
Generated documents: {len(generated_documents)} total (generated only)
Client documents: {len(client_documents)} total""")


## Run Attack


In [ ]:
# Function to turn document into list of keywords

import re
import nltk

from typing import List
from nltk.corpus import stopwords, words
from nltk.stem.porter import PorterStemmer
from collections import Counter

def process_document(document: str) -> set:
    nltk.download('stopwords', quiet=True)
    english_stopwords = stopwords.words('english')

    words = list(map(str.lower, re.findall(r'\b\w+\b', document)))
    # Keep stems of words that are between 2 and 20 characters and not stopwords
    ps = PorterStemmer()
    stems = set(ps.stem(word) for word in words if word.lower() not in english_stopwords and 2 <= len(word) <= 20 and word.isalpha())

    return stems

def process_documents(documents: List[str]) -> List[set]:
    ret = []
    for document in documents:
        ret.append(process_document(document))
    return ret



attacker_set = process_documents(attacker_documents)
generated_set = process_documents(generated_documents)

attacker_enhanced_set = process_documents(attacker_documents + generated_documents)

client_set = process_documents(client_documents)

print(f"""Attacker enhanced set: {len(attacker_enhanced_set)}
Attacker set: {len(attacker_set)}
Client set: {len(client_set)}""")

In [ ]:
# Create cooccurrence matrix
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np


def create_cooccurrence_matrix(document_set):
    print(f"create_cooccurrence_matrix: {len(document_set)} documents")
    mlb = MultiLabelBinarizer()
    cooccurrence_matrix = mlb.fit_transform(document_set)
    cooccurrence_matrix = cooccurrence_matrix.T @ cooccurrence_matrix
    keyword_universe = list(mlb.classes_)
    # print(cooccurrence_matrix)
    # print(keyword_universe)
    return list(zip(np.diagonal(cooccurrence_matrix) / len(document_set), keyword_universe))

def create_occurrence_matrix(document_set):
    print(f"create_occurrence_matrix: {len(document_set)} documents")
    counter = Counter()
    for doc in document_set:
        counter.update(doc)
    keyword_universe, occurrence_matrix = zip(*counter.items())
    occurrence_matrix = np.array(occurrence_matrix)

    return list(zip(occurrence_matrix / len(document_set), keyword_universe))

attacker = create_occurrence_matrix(attacker_set)
attacker.sort(reverse=True)
print(attacker)

attacker_enhanced = create_occurrence_matrix(attacker_enhanced_set)
attacker_enhanced.sort(reverse=True)
print(attacker_enhanced)

client = create_occurrence_matrix(client_set)
client.sort(reverse=True)
print(client)



# attacker = create_cooccurrence_matrix(attacker_set)
# attacker.sort(reverse=True)
# # print(attacker)

# client = create_cooccurrence_matrix(client_set)
# client.sort(reverse=True)

# print(attacker[:10])
# print(client[:10])




create_occurrence_matrix: 200 documents
[(np.float64(0.305), 'thank'), (np.float64(0.29), 'subject'), (np.float64(0.29), 'cc'), (np.float64(0.26), 'enron'), (np.float64(0.245), 'know'), (np.float64(0.23), 'pleas'), (np.float64(0.215), 'pm'), (np.float64(0.19), 'get'), (np.float64(0.185), 'let'), (np.float64(0.17), 'would'), (np.float64(0.16), 'ect'), (np.float64(0.15), 'work'), (np.float64(0.15), 'time'), (np.float64(0.15), 'go'), (np.float64(0.14), 'think'), (np.float64(0.14), 'like'), (np.float64(0.14), 'hou'), (np.float64(0.135), 'need'), (np.float64(0.135), 'kay'), (np.float64(0.13), 'one'), (np.float64(0.13), 'new'), (np.float64(0.13), 'make'), (np.float64(0.125), 'deal'), (np.float64(0.125), 'com'), (np.float64(0.12), 'want'), (np.float64(0.11), 'look'), (np.float64(0.11), 'day'), (np.float64(0.105), 'see'), (np.float64(0.105), 'good'), (np.float64(0.105), 'also'), (np.float64(0.1), 'well'), (np.float64(0.1), 'question'), (np.float64(0.1), 'chang'), (np.float64(0.095), 'us'), (np

## Evaluation

In [ ]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union if union != 0 else 0

a = set(k for v, k in attacker)
ae = set(k for v, k in attacker_enhanced)
c = set(k for v, k in client)

print(f"Jaccard Attacker and Attacker Enhanced: {jaccard_similarity(a, ae)}")
print(f"Jaccard Attacker and Client: {jaccard_similarity(a, c)}")
print(f"Jaccard Client and Attacker Enhanced: {jaccard_similarity(ae, c)}")

Jaccard Attacker and Attacker Enhanced: 0.7129406084017383
Jaccard Attacker and Client: 0.2537493952588292
Jaccard Client and Attacker Enhanced: 0.3047651309846432


In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np


mpl.rcParams['figure.dpi'] = 1200

# Given data
list1 = client
list2 = attacker_enhanced

# Extract words in order of list1, but only plot top 100
words_ordered = [word for _, word in list1][:100]

# Create dictionaries for quick lookup
occurrences1 = {word: count for count, word in list1}
occurrences2 = {word: count for count, word in list2}

# Ensure all words in list1 are included in occurrences2, defaulting to zero if missing
occurrences2 = {word: occurrences2.get(word, 0) for word in words_ordered}

# Extract counts in order of list1
counts1 = [occurrences1[word] for word in words_ordered]
counts2 = [occurrences2[word] for word in words_ordered]

# Plot
x = np.arange(len(words_ordered))  # x locations
width = 0.4  # Bar width

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, counts1, width, label='Client (Ground Truth)')
rects2 = ax.bar(x + width/2, counts2, width, label='Augmented with gpt-4o-mini')

# Labels, titles, and legend
ax.set_xlabel('Words')
ax.set_ylabel('Occurrences')
ax.set_title(f'Enron top 100 keywords (attacker {DOCUMENT_COUNT_TO_ATTACKER} docs, client {DOCUMENT_COUNT_TO_CLIENT} docs)')
ax.set_xticks(x)
ax.set_xticklabels(words_ordered, rotation=45, ha="right")
ax.tick_params(axis='x', labelsize=4)
ax.legend()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np


mpl.rcParams['figure.dpi'] = 1200

# Given data
list1 = attacker
list2 = attacker_enhanced

# Extract words in order of list1, but only plot top 100
words_ordered = [word for _, word in list1][:100]

# Create dictionaries for quick lookup
occurrences1 = {word: count for count, word in list1}
occurrences2 = {word: count for count, word in list2}

# Ensure all words in list1 are included in occurrences2, defaulting to zero if missing
occurrences2 = {word: occurrences2.get(word, 0) for word in words_ordered}

# Extract counts in order of list1
counts1 = [occurrences1[word] for word in words_ordered]
counts2 = [occurrences2[word] for word in words_ordered]

# Plot
x = np.arange(len(words_ordered))  # x locations
width = 0.4  # Bar width

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, counts1, width, label='Client (Ground Truth)')
rects2 = ax.bar(x + width/2, counts2, width, label='Augmented with gpt-4o-mini')

# Labels, titles, and legend
ax.set_xlabel('Words')
ax.set_ylabel('Occurrences')
ax.set_title(f'Enron top 100 keywords (attacker {DOCUMENT_COUNT_TO_ATTACKER} docs, client {DOCUMENT_COUNT_TO_CLIENT} docs)')
ax.set_xticks(x)
ax.set_xticklabels(words_ordered, rotation=45, ha="right")
ax.tick_params(axis='x', labelsize=4)
ax.legend()

# Show the plot
plt.show()
